# Collusion Between Static and Moving Object 

We are gonna talk about collusion theory in pygame  and how we can avoid bugs . 

First thing to mention about is , you have to use rectangles to detect collusions . Easiest way to detect rectangle collusions is `rect1.colliderect(Rect2)` function that returns boolean.  

> Problem of this approach is , colliderect function doesnt tell you which side the two objects have been colliding with each other.  We need to know if the projectile object is coming from left bottom or right top to simulate bounce mechanics. 

<img src="img/directions_of_rect_collidepoints_method.jpg"  width = "600px">


Secondly we can use `rect1.collidepoints((x,y))` to detect collusions . This method detects if the point `(x,y)` collides with the rectangle. We can use 8 main directions for this problem but this is also a bug prone approach.  

<img src="img/badCollusion1.jpg" width="600px"> 

> Colliding middlepoints cause a bug that the colliding object didnt recognized when it is coming straight to the corners 

<img src="img/badCollusion2.jpg" width="600px">  

> Colliding corners also not an ideal approach to the problem because you cant detect which direction the object is coming from 

<img src="img/badCollusion3-1.jpg" width="600px">  
<img src="img/badCollusion3-2.jpg" width="600px"> 

> Combining the approaches above doesnt solve our problem totally either.When the object is a bit small that can jam in between 2 detecting points , colliding object didnt recognized properly. In Addition object's colliding points could aint match detecting points like in the second visual . Also you have to code so much desicion statements for this kind of detection. 

### Solution for the Proper Collusion 

<img src="img/goodCollusion0.jpg" width="600px">  
<img src="img/goodCollusion1.jpg" width="600px"> 

Proper collusion algorithm : 
1. Check if the two  rectangles are collisioning with each other 
2. Check if the first objects bottom and second objects top are laying together. 

__IMPORTANT NOTE__ : We can check the point in every frame and that is an discrete value not an continuous value. Because of that we can miss the absolute laying points (which is top-bottom = 0) . Considering this , we need to add 'collision_tolerance' to the equation 

<img src="img/why_taking_absolute_values_of_directions_difference.jpg" width="600px">

__IMPORTANT SECOND NOTE__ : We also need to take  absolute value of the difference . Otherwise other detecting points can be exceed from our condition and cause lots of bugs 


<img src="img/problem_collusion_of_moving_2_objects.jpg" width="600px">

__IMPORTANT THIRD NOTE__ : Also when both of the colliding objects are moving , multiplying the speed by -1 can be triggered multiple times . This also cause a bug that colliding object continues it's way by going through the second object. We can pretend this by adding `if velocity's direction is this` to our control statements  

In [1]:
import pygame   
import random

WHITE = (255,255,255) 
BLACK = (0,0,0) 
RED = (255,0,0) 
BLUE = (0,0,255) 
GREEN = (0,255,0) 

WIDTH , HEIGHT = 600 , 600

class Rect2D(pygame.sprite.Sprite): 
    def __init__(self, width , height , pos_x , pos_y , velocity_x = 0 , velocity_y = 0 , has_collusion = False) -> None: 
        global WHITE
        super().__init__()  
        self.hasCollusion = has_collusion
        self.image = pygame.Surface([width , height])  
        self.color = random.choice([WHITE , RED , GREEN , BLUE])
        self.image.fill(self.color) 
        self.isCollusion = False  
        self.colDirection = "NONE"
        self.rect = self.image.get_rect() 
        self.rect.centerx , self.rect.centery = pos_x , pos_y 
        self.vx = velocity_x 
        self.vy = velocity_y  
        pygame.draw.rect(self.image , self.color , self.rect)

    def collusion(self): 
        if  self.isCollusion : 
            if self.colDirection == "TOP" and self.vy < 0 : 
                self.vy *= -1  
            if self.colDirection == "BOTTOM" and self.vy > 0 : 
                self.vy *= -1 
            if self.colDirection == "LEFT" and self.vx < 0 :  
                self.vx *= -1 
            if self.colDirection == "RIGHT" and self.vx > 0 : 
                self.vx *= -1  

    def update(self): 
        
        self.collusion()
        
        self.rect.centerx += self.vx 
        self.rect.centery += self.vy 

        if self.rect.right >= WIDTH or self.rect.left <= 0 :  
            self.vx *= -1   

        if self.rect.bottom >= HEIGHT or self.rect.top <= 0 : 
            self.vy *= -1  

        self.isCollusion = False  
        self.colDirection = "NONE"


pygame.init() 
screen = pygame.display.set_mode( (WIDTH , HEIGHT)) 
rectGroup = pygame.sprite.Group() 
rect1 = Rect2D(50 , 50 ,300 , 200 , 7 , 7 , True) 
rectGroup.add(rect1)   
rect2 = Rect2D(100 , 30 , WIDTH//2  , HEIGHT//2 , 3 , 3 , True) 
rectGroup.add(rect2)
clock = pygame.time.Clock()   


def collusionByObjectMovement(COLLUSION_TOLERANCE = 10): 
    sprite_list = rectGroup.sprites()   
    print(sprite_list)
    for index in range(len(sprite_list)) : 
        for index2 in  range(len(sprite_list)) : 
            if index == index2 : 
                continue
            obj1 = sprite_list[index] 
            obj2 = sprite_list[index2]
            if obj1.rect.colliderect(obj2.rect):  
                if abs(obj1.rect.left - obj2.rect.right) <= COLLUSION_TOLERANCE : 
                    
                    if obj1.hasCollusion :
                        obj1.isCollusion = True  
                        obj1.colDirection = "LEFT"  
 
                
                if abs(obj1.rect.right - obj2.rect.left) <= COLLUSION_TOLERANCE: 
                    
                    if obj1.hasCollusion :
                        obj1.isCollusion = True 
                        obj1.colDirection = "RIGHT"  


                if abs(obj1.rect.top - obj2.rect.bottom) <= COLLUSION_TOLERANCE: 
                    if obj1.hasCollusion :    
                        obj1.isCollusion = True 
                        obj1.colDirection = "TOP" 

   

                if abs(obj1.rect.bottom - obj2.rect.top) <= COLLUSION_TOLERANCE: 
                    
                    if obj1.hasCollusion :
                        obj1.isCollusion = True 
                        obj1.colDirection = "BOTTOM" 
            
                



running = True 
while running : 
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT: 
            running = False 
            exit() 
            pygame.quit() 

    
    screen.fill(BLACK) 
    rectGroup.draw(screen)  
    collusionByObjectMovement()
    rectGroup.update()  
    pygame.display.flip()
    clock.tick(60)






pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
[<Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>]
[<Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>]
[<Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>]
[<Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>]
[<Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>]
[<Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>]
[<Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>]
[<Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>]
[<Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>]
[<Rect2D Sprite(in 1 groups)>, <Rect2D Sprite(in 1 groups)>, <Rect2D

error: display Surface quit

: 